In [1]:
%pip install annoy
%pip install lightgbm scikit-learn


StatementMeta(, bc2c5a25-4507-4d1f-8c83-7c0b0ffd88fa, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=77476 sha256=9b999553dc7756462a5f1452bfcbca9751359a6a641885405b017fa1d503bcdb
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [5]:
# ============================================
# CELL 1 — CONFIG & IMPORTS
# ============================================
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql import Row
import numpy as np

# Table / path constants
GOLD_LABEL_TABLE    = "gold_ml_training_set"
USER_EMB_TABLE      = "gold_two_tower_user_emb"
ITEM_EMB_TABLE      = "gold_two_tower_item_emb"
INTERACTIONS_TABLE  = "gold_user_interactions"
CATALOG_TABLE       = "gold_ml_catalog"
MODEL_PATH          = "Files/models/gbt_physics_baseline"

# Audio feature columns
AUDIO_FEATURES = [
    "danceability", "energy", "key", "loudness", "mode", "speechiness",
    "acousticness", "instrumentalness", "liveness", "valence", "tempo"
]

print("✅ Config loaded.")


StatementMeta(, bc2c5a25-4507-4d1f-8c83-7c0b0ffd88fa, 13, Finished, Available, Finished)

✅ Config loaded.


In [39]:
# ============================================
# BUILD UNIFIED AUDIO + METADATA CATALOG
# gold_ml_catalog
# ============================================

from pyspark.sql.functions import col

audio_raw  = spark.table("silver_audio_features_raw")
tracks_raw = spark.table("silver_tracks_raw")
ml_public  = spark.table("silver_ml_training_set")

final_cols = ["spotify_id", "track_name", "artist_name"] + AUDIO_FEATURES

# --------------------------------------------
# 1) User-side (private) high-quality metadata
# --------------------------------------------
df_user = (
    audio_raw.join(tracks_raw, "spotify_id")
        .withColumnRenamed("artist_recco_id", "artist_id")
        .select(final_cols)
)

# --------------------------------------------
# 2) Public ML tracks (fallback metadata)
# --------------------------------------------
df_public = (
    ml_public
        .withColumn("artist_name", col("artists"))
        .select(final_cols)
)

# Only include public tracks that are NOT in df_user
df_public_new = df_public.join(df_user, "spotify_id", "left_anti")

# --------------------------------------------
# 3) Build unified catalog
# --------------------------------------------
catalog_unified = df_user.unionByName(df_public_new)

# Remove rows missing audio features
catalog_clean = catalog_unified.na.drop(subset=AUDIO_FEATURES)

# Save
catalog_clean.write.mode("overwrite").format("delta").saveAsTable(CATALOG_TABLE)

total = spark.table(CATALOG_TABLE).count()
print(f"✅ Catalog saved: {CATALOG_TABLE}")
print(f"📊 Total songs: {total:,}")


StatementMeta(, 49a05bbd-c6a8-4416-a4f5-03f18f92bc70, 47, Finished, Available, Finished)

✅ Catalog saved: gold_ml_catalog
📊 Total songs: 1,239,363


In [40]:
# ============================================
# CELL 2 — BUILD INTERACTIONS TABLE (ONCE)
# ============================================
from pyspark.sql.functions import col

recent = spark.table("silver_recently_played").select(col("spotify_user_id"), col("track_id"))
saved  = spark.table("silver_saved_tracks").select(col("spotify_user_id"), col("track_id"))
playlist = spark.table("silver_playlist_tracks").select(col("spotify_user_id"), col("track_id"))
top = spark.table("silver_top_tracks").select(col("spotify_user_id"), col("track_id"))

interactions = recent.union(saved).union(playlist).union(top).dropna().distinct()

# Persist to table (overwrite)
interactions.write.mode("overwrite").saveAsTable(INTERACTIONS_TABLE)

print(f"✅ Interactions table saved: {INTERACTIONS_TABLE}")
print(f"📦 Rows: {interactions.count():,}")


StatementMeta(, 49a05bbd-c6a8-4416-a4f5-03f18f92bc70, 48, Finished, Available, Finished)

✅ Interactions table saved: gold_user_interactions
📦 Rows: 31,099


In [41]:
# ============================================
# CELL 3 — INDEX USERS & ITEMS
# ============================================
from pyspark.sql.functions import row_number

inter = spark.table(INTERACTIONS_TABLE).select("spotify_user_id", "track_id").distinct()

# User index
w_user = Window.orderBy("spotify_user_id")
user_index = (
    inter.select("spotify_user_id").distinct()
         .withColumn("user_idx", row_number().over(w_user) - 1)
)

# Item index
w_item = Window.orderBy("track_id")
item_index = (
    inter.select("track_id").distinct()
         .withColumn("item_idx", row_number().over(w_item) - 1)
)

num_users = user_index.count()
num_items = item_index.count()

print(f"👥 Users: {num_users:,}")
print(f"🎵 Items: {num_items:,}")

indexed = (
    inter.join(user_index, "spotify_user_id")
         .join(item_index, "track_id")
         .select("spotify_user_id","track_id","user_idx","item_idx")
)

indexed.cache()
print(f"📦 Indexed interactions: {indexed.count():,}")


StatementMeta(, 49a05bbd-c6a8-4416-a4f5-03f18f92bc70, 49, Finished, Available, Finished)

👥 Users: 27
🎵 Items: 27,768
📦 Indexed interactions: 31,099


In [44]:
# ============================================
# CELL 4 — TRAIN 2-TOWER MODEL (NUMPY BPR)
# ============================================
import numpy as np

# Recompute num_users / num_items if needed
num_users = indexed.agg(F.max("user_idx")).collect()[0][0] + 1
num_items = indexed.agg(F.max("item_idx")).collect()[0][0] + 1

print(f"👥 num_users = {num_users}")
print(f"🎵 num_items = {num_items}")

# Collect interactions (sample if too large)
indexed_pd = indexed.toPandas()
user_indices = indexed_pd["user_idx"].values.astype(np.int32)
item_indices = indexed_pd["item_idx"].values.astype(np.int32)
n_interactions = len(user_indices)
print(f"📦 Interactions loaded: {n_interactions:,}")

# HYPERPARAMS
embedding_dim = 256
learning_rate = 0.05
reg_lambda = 0.001
num_epochs = 5
batch_size = 4096

# Initialize embeddings
rng = np.random.default_rng(42)
user_emb_matrix = 0.01 * rng.standard_normal((num_users, embedding_dim), dtype=np.float32)
item_emb_matrix = 0.01 * rng.standard_normal((num_items, embedding_dim), dtype=np.float32)

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

# Training loop (BPR)
for epoch in range(1, num_epochs + 1):
    perm = rng.permutation(n_interactions)
    user_indices_sh = user_indices[perm]
    item_indices_sh = item_indices[perm]

    total_loss = 0.0
    n_steps = 0

    for start in range(0, n_interactions, batch_size):
        end = min(start + batch_size, n_interactions)
        u_batch = user_indices_sh[start:end]
        i_batch = item_indices_sh[start:end]
        batch_len = len(u_batch)
        if batch_len == 0:
            continue

        j_batch = rng.integers(low=0, high=num_items, size=batch_len, dtype=np.int32)

        u_vecs = user_emb_matrix[u_batch]
        i_vecs = item_emb_matrix[i_batch]
        j_vecs = item_emb_matrix[j_batch]

        x_ui = np.sum(u_vecs * i_vecs, axis=1)
        x_uj = np.sum(u_vecs * j_vecs, axis=1)
        x_uij = x_ui - x_uj
        sig = sigmoid(x_uij)
        loss_vec = -np.log(sig + 1e-10)
        loss = np.mean(loss_vec) + reg_lambda * (
            np.mean(np.sum(u_vecs*u_vecs, axis=1)) +
            np.mean(np.sum(i_vecs*i_vecs, axis=1)) +
            np.mean(np.sum(j_vecs*j_vecs, axis=1))
        )
        total_loss += loss
        n_steps += 1

        grad_factor = (sig - 1.0).reshape(-1, 1)
        grad_u = grad_factor * (i_vecs - j_vecs) + reg_lambda * u_vecs
        grad_i = grad_factor * u_vecs + reg_lambda * i_vecs
        grad_j = -grad_factor * u_vecs + reg_lambda * j_vecs

        user_emb_matrix[u_batch]  -= learning_rate * grad_u
        item_emb_matrix[i_batch]  -= learning_rate * grad_i
        item_emb_matrix[j_batch]  -= learning_rate * grad_j

    avg_loss = total_loss / max(1, n_steps)
    print(f"📉 Epoch {epoch}/{num_epochs} - avg BPR loss: {avg_loss:.4f}")

print("✅ 2-tower (NumPy BPR) training complete.")


StatementMeta(, 49a05bbd-c6a8-4416-a4f5-03f18f92bc70, 52, Finished, Available, Finished)

👥 num_users = 27
🎵 num_items = 27768
📦 Interactions loaded: 31,099
📉 Epoch 1/5 - avg BPR loss: 0.6933
📉 Epoch 2/5 - avg BPR loss: 0.6930
📉 Epoch 3/5 - avg BPR loss: 0.6927
📉 Epoch 4/5 - avg BPR loss: 0.6924
📉 Epoch 5/5 - avg BPR loss: 0.6921
✅ 2-tower (NumPy BPR) training complete.


In [45]:
# ============================================
# CELL 5 — EXPORT EMBEDDINGS TO SPARK
# ============================================
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql import Row

embedding_dim = user_emb_matrix.shape[1]
print(f"🧠 Exporting embeddings of dim {embedding_dim}")

# USER EMBEDDINGS
user_index_pd = user_index.select("spotify_user_id","user_idx").toPandas()
user_index_pd["vector"] = list(user_emb_matrix[user_index_pd["user_idx"].values])
user_rows = [
    Row(spotify_user_id=row.spotify_user_id,
        vector=[float(x) for x in row.vector])
    for row in user_index_pd.itertuples()
]
user_emb_spark = spark.createDataFrame(user_rows)

# ITEM EMBEDDINGS (only for items seen in interactions)
item_index_pd = item_index.select("track_id","item_idx").toPandas()
item_index_pd["vector"] = list(item_emb_matrix[item_index_pd["item_idx"].values])
item_rows = [
    Row(spotify_id=row.track_id,
        vector=[float(x) for x in row.vector])
    for row in item_index_pd.itertuples()
]
item_emb_spark = spark.createDataFrame(item_rows)

# TAG ITEMS AS OCEAN vs FRIENDS
audio_private = spark.table("silver_audio_features_raw") \
    .select("spotify_id").distinct() \
    .withColumn("source", F.lit("friends"))

public_ml = spark.table("silver_ml_training_set") \
    .select(F.col("spotify_id").cast("string").alias("spotify_id")) \
    .distinct() \
    .withColumn("source", F.lit("ocean"))

source_map = audio_private.unionByName(public_ml).dropDuplicates(["spotify_id"])

item_emb_spark = item_emb_spark.join(source_map, "spotify_id", "left").fillna({"source":"unknown"})

# SAVE TO TABLES
user_emb_spark.write.mode("overwrite").saveAsTable(USER_EMB_TABLE)
item_emb_spark.write.mode("overwrite").saveAsTable(ITEM_EMB_TABLE)

print(f"✅ User embeddings saved to:  {USER_EMB_TABLE}")
print(f"✅ Item embeddings saved to:  {ITEM_EMB_TABLE}")
print(f"👥 Users with embeddings: {user_emb_spark.count():,}")
print(f"🎵 Items with embeddings: {item_emb_spark.count():,}")


StatementMeta(, 49a05bbd-c6a8-4416-a4f5-03f18f92bc70, 53, Finished, Available, Finished)

🧠 Exporting embeddings of dim 256
✅ User embeddings saved to:  gold_two_tower_user_emb
✅ Item embeddings saved to:  gold_two_tower_item_emb
👥 Users with embeddings: 27
🎵 Items with embeddings: 27,768


In [42]:
# ============================================
# CELL 6 — BUILD gold_ml_training_set (POSITIVE LABELS)
# ============================================
from pyspark.sql.functions import col, lit

rp = spark.table("silver_recently_played").select(col("spotify_user_id"), col("track_id").alias("spotify_id"))
saved = spark.table("silver_saved_tracks").select(col("spotify_user_id"), col("track_id").alias("spotify_id"))
pl = spark.table("silver_playlist_tracks").select(col("spotify_user_id"), col("track_id").alias("spotify_id"))
top = spark.table("silver_top_tracks").select(col("spotify_user_id"), col("track_id").alias("spotify_id"))

gold = rp.union(saved).union(pl).union(top).distinct().withColumn("label", lit(1))
gold.write.mode("overwrite").saveAsTable(GOLD_LABEL_TABLE)

print("✅ Created gold_ml_training_set")
print("📦 Rows:", gold.count())

from pyspark.sql.functions import col


StatementMeta(, 49a05bbd-c6a8-4416-a4f5-03f18f92bc70, 50, Finished, Available, Finished)

✅ Created gold_ml_training_set
📦 Rows: 31107


In [46]:
# ============================================
# CELL 7 — REBUILD ITEM EMBEDDINGS (HYBRID: 2-TOWER + AUDIO)
# ============================================
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit

EMB_DIM = 256
PAD_SIZE = EMB_DIM - len(AUDIO_FEATURES)
print(f"Using hybrid embeddings with dim = {EMB_DIM} (audio + padding, or 2-tower).")

catalog = spark.table(CATALOG_TABLE).select("spotify_id", *AUDIO_FEATURES)
print("📦 Catalog rows:", catalog.count())

existing_emb = spark.table(ITEM_EMB_TABLE)
print("📦 Existing 2-tower item embeddings:", existing_emb.count())

inter = spark.table(INTERACTIONS_TABLE).select(col("track_id").alias("spotify_id")).distinct()
friends_ids = inter.withColumn("source", lit("friends"))

base = (
    catalog
    .join(existing_emb.select("spotify_id", "vector"), "spotify_id", "left")
    .join(friends_ids, "spotify_id", "left")
)

base = base.withColumn(
    "source",
    F.when(col("source").isNull(), lit("ocean")).otherwise(col("source"))
)

audio_as_vec = F.array(
    *[col(c).cast("float") for c in AUDIO_FEATURES] +
    [F.lit(0.0).cast("float")] * PAD_SIZE
)

hybrid = base.withColumn(
    "vector",
    F.when(col("vector").isNotNull(), col("vector")).otherwise(audio_as_vec)
)

final_item_emb = hybrid.select("spotify_id", "vector", "source")
print("✅ Hybrid item embeddings built.")
print("   Total rows:", final_item_emb.count())
final_item_emb.groupBy("source").count().show()

final_item_emb.write.mode("overwrite").saveAsTable(ITEM_EMB_TABLE)
print(f"💾 Overwrote {ITEM_EMB_TABLE} with hybrid embeddings.")


StatementMeta(, 49a05bbd-c6a8-4416-a4f5-03f18f92bc70, 54, Finished, Available, Finished)

Using hybrid embeddings with dim = 256 (audio + padding, or 2-tower).
📦 Catalog rows: 1239363
📦 Existing 2-tower item embeddings: 27768
✅ Hybrid item embeddings built.
   Total rows: 1239363
+-------+-------+
| source|  count|
+-------+-------+
|friends|  23238|
|  ocean|1216125|
+-------+-------+

💾 Overwrote gold_two_tower_item_emb with hybrid embeddings.


In [8]:
from annoy import AnnoyIndex
import numpy as np
import joblib
import os
from pyspark.sql.functions import col

# Correct persistent Fabric directory
ANNOY_DIR = "/lakehouse/default/Files/annoy"
os.makedirs(ANNOY_DIR, exist_ok=True)

item_df = spark.table(ITEM_EMB_TABLE).select("spotify_id", "vector", "source")

friends_pd = item_df.filter(col("source") == "friends").toPandas()
ocean_pd   = item_df.filter(col("source") == "ocean").toPandas()

dim = len(friends_pd.iloc[0].vector)

# -------------------------
# Build FRIENDS index
# -------------------------
friends_index = AnnoyIndex(dim, 'angular')
friends_map = {}

for idx, row in enumerate(friends_pd.itertuples()):
    friends_index.add_item(idx, np.array(row.vector, dtype="float32"))
    friends_map[idx] = row.spotify_id

friends_index.build(20)
friends_index.save(f"{ANNOY_DIR}/friends_index.ann")
joblib.dump(friends_map, f"{ANNOY_DIR}/friends_map.pkl")

# -------------------------
# Build OCEAN index
# -------------------------
ocean_index = AnnoyIndex(dim, 'angular')
ocean_map = {}

for idx, row in enumerate(ocean_pd.itertuples()):
    ocean_index.add_item(idx, np.array(row.vector, dtype="float32"))
    ocean_map[idx] = row.spotify_id

ocean_index.build(20)
ocean_index.save(f"{ANNOY_DIR}/ocean_index.ann")
joblib.dump(ocean_map, f"{ANNOY_DIR}/ocean_map.pkl")

print("✅ Annoy indices saved to /lakehouse/default/Files/annoy/")


StatementMeta(, bc2c5a25-4507-4d1f-8c83-7c0b0ffd88fa, 16, Finished, Available, Finished)

✅ Annoy indices saved to /lakehouse/default/Files/annoy/


In [6]:
# ============================================
# CELL 8 — TRAIN LIGHTGBM RE-RANKER (HARD NEGATIVES)
# ============================================
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from collections import defaultdict
from annoy import AnnoyIndex
from datetime import datetime
import numpy as np
import joblib
import os
import lightgbm as lgb

# ------------------------------------------------
# CONFIG
# ------------------------------------------------
TRAIN_USER_LIMIT = 3000    # how many users to sample for training
K_PER_USER       = 200     # how many neighbors per user

print("⚙️ Training LightGBM with HARD negatives")
print(f"   Users sampled: {TRAIN_USER_LIMIT}, K per user: {K_PER_USER}")

# ------------------------------------------------
# 1) Load user embeddings (only users we have vectors for)
# ------------------------------------------------
user_emb_spark = spark.table(USER_EMB_TABLE)   # spotify_user_id, vector
gold_users = spark.table(GOLD_LABEL_TABLE).select("spotify_user_id").distinct()

train_users_spark = (
    gold_users.join(user_emb_spark, "spotify_user_id", "inner")
              .limit(TRAIN_USER_LIMIT)
)

train_users_pd = train_users_spark.toPandas()
print(f"👥 Training users with embeddings: {len(train_users_pd):,}")

if train_users_pd.empty:
    raise Exception("No users with embeddings found for training.")

# ------------------------------------------------
# 2) Load item embeddings + build Annoy index
# ------------------------------------------------
item_emb_spark = spark.table(ITEM_EMB_TABLE).select("spotify_id", "vector")
item_pd = item_emb_spark.toPandas()

print(f"🎵 Items with embeddings: {len(item_pd):,}")

dim = len(item_pd.iloc[0].vector)
ann = AnnoyIndex(dim, 'angular')

for idx, row in enumerate(item_pd.itertuples()):
    ann.add_item(idx, np.array(row.vector, dtype="float32"))

print("⚙️ Building Annoy index for items...")
ann.build(20)
print("✅ Annoy index built.")

# ------------------------------------------------
# 3) Build positive map: which tracks each user actually played
# ------------------------------------------------
pos_pd = (
    spark.table(GOLD_LABEL_TABLE)
         .select("spotify_user_id", "spotify_id")
         .distinct()
         .toPandas()
)

pos_map = defaultdict(set)
for row in pos_pd.itertuples():
    pos_map[row.spotify_user_id].add(row.spotify_id)

print(f"📦 Positive interactions (rows): {len(pos_pd):,}")

# ------------------------------------------------
# 4) Generate (user, item, label) pairs from ANN neighbors
# ------------------------------------------------
training_rows = []  # (spotify_user_id, spotify_id, label)

for u in train_users_pd.itertuples():
    u_id = u.spotify_user_id
    u_vec = np.array(u.vector, dtype="float32")

    # get ANN neighbors for this user
    idxs = ann.get_nns_by_vector(u_vec, K_PER_USER)

    user_pos_set = pos_map[u_id]

    for idx in idxs:
        sid = item_pd.iloc[idx].spotify_id
        label = 1 if sid in user_pos_set else 0
        training_rows.append((u_id, sid, int(label)))

print(f"🧱 Generated training pairs: {len(training_rows):,}")

if not training_rows:
    raise Exception("No training pairs generated. Check your embeddings and GOLD_LABEL_TABLE.")

# ------------------------------------------------
# 5) Convert to Spark and join audio features
# ------------------------------------------------
training_spark = spark.createDataFrame(
    training_rows,
    ["spotify_user_id", "spotify_id", "label"]
)

catalog = spark.table(CATALOG_TABLE).select("spotify_id", *AUDIO_FEATURES)

train_joined = (
    training_spark.join(catalog, "spotify_id", "inner")
)

print(f"📦 Train rows after joining audio features: {train_joined.count():,}")

# ------------------------------------------------
# 6) Spark → Pandas → LightGBM
# ------------------------------------------------
pdf = train_joined.select(AUDIO_FEATURES + ["label"]).toPandas()

X = pdf[AUDIO_FEATURES].values.astype("float32")
y = pdf["label"].values.astype("int32")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"🧪 Train size: {len(X_train):,}, Test size: {len(X_test):,}")

# ------------------------------------------------
# 7) Train LightGBM (Fabric-safe: no kwargs in fit)
# ------------------------------------------------
model_lgb = lgb.LGBMClassifier(
    objective="binary",
    learning_rate=0.05,
    num_leaves=63,
    n_estimators=300,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=0.1,
    reg_alpha=0.0,
)

print("🚀 Training LightGBM re-ranker (hard negatives)...")
model_lgb.fit(X_train, y_train)

# ------------------------------------------------
# 8) Evaluate
# ------------------------------------------------
y_prob = model_lgb.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_prob)
acc = accuracy_score(y_test, y_pred)

print(f"\n🎯 LightGBM Accuracy (hard negatives): {acc*100:.2f}%")
print(f"📈 LightGBM ROC AUC (hard negatives):  {auc:.3f}")

# ------------------------------------------------
# 9) Save model  (FABRIC-SAFE)
# ------------------------------------------------

MODEL_DIR = "/lakehouse/default/Files/models"
os.makedirs(MODEL_DIR, exist_ok=True)

LGBM_MODEL_PATH = f"{MODEL_DIR}/lgbm_re_ranker.pkl"
joblib.dump(model_lgb, LGBM_MODEL_PATH)

print(f"💾 LightGBM model saved to: {LGBM_MODEL_PATH}")

MODEL_VERSION = f"lgbm_v_{datetime.now().strftime('%Y%m%d_%H%M')}"
MODEL_TRAINED_AT = datetime.now().isoformat()

print("📌 MODEL VERSION:", MODEL_VERSION)
print("📌 TRAINED AT:", MODEL_TRAINED_AT)

# Save metadata into delta table
metadata_df = spark.createDataFrame(
    [(MODEL_VERSION, MODEL_TRAINED_AT)],
    ["model_version", "trained_at"]
)

metadata_df.write.mode("overwrite").saveAsTable("gold_model_metadata")

StatementMeta(, bc2c5a25-4507-4d1f-8c83-7c0b0ffd88fa, 14, Finished, Available, Finished)

⚙️ Training LightGBM with HARD negatives
   Users sampled: 3000, K per user: 200
👥 Training users with embeddings: 27
🎵 Items with embeddings: 1,239,363
⚙️ Building Annoy index for items...
✅ Annoy index built.
📦 Positive interactions (rows): 31,107
🧱 Generated training pairs: 5,400
📦 Train rows after joining audio features: 7,521
🧪 Train size: 6,016, Test size: 1,505
🚀 Training LightGBM re-ranker (hard negatives)...
[LightGBM] [Info] Number of positive: 1401, number of negative: 4615
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022528 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2304
[LightGBM] [Info] Number of data points in the train set: 6016, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.232879 -> initscore=-1.192126
[LightGBM] [Info] Start training from score -1.192126



🎯 LightGBM Accuracy (hard negatives): 81.99%
📈 LightGBM ROC AUC (hard negatives):  0.792
💾 LightGBM model saved to: /lakehouse/default/Files/models/lgbm_re_ranker.pkl
📌 MODEL VERSION: lgbm_v_20251125_0647
📌 TRAINED AT: 2025-11-25T06:47:58.734444


In [7]:
import os

print(os.path.exists("/lakehouse/default/Files/models/lgbm_re_ranker.pkl"))


StatementMeta(, bc2c5a25-4507-4d1f-8c83-7c0b0ffd88fa, 15, Finished, Available, Finished)

True
